In [1]:
# import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.python.framework import ops
ops.reset_default_graph()

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
Xdf_raw = pd.read_csv("IndoorFingerprint.csv").drop("Y", axis=1)
Ydf_raw = pd.read_csv("IndoorFingerprint.csv").drop("X", axis=1)

In [3]:
cardinal_numbers = 4;
atribute_numbers = Xdf_raw.shape[1]-1

# Valores para el dataset que predice X
Xtrx, Xtex, ytrx, ytex = train_test_split(
Xdf_raw.iloc[:, 1:], Xdf_raw["X"], test_size=0.2, random_state=0)
train_size = len(Xtrx)
test_size = len(Xtex)

# Valores para el dataset que predice Y
Xtry, Xtey, ytry, ytey = train_test_split(
Ydf_raw.iloc[:, 1:], Ydf_raw["Y"], test_size=0.2, random_state=0)

# Se normalizan los datos para ambos

scaler = preprocessing.StandardScaler()
Xtrx.iloc[:, 4:atribute_numbers] = scaler.fit_transform(Xtrx.iloc[:, 4:atribute_numbers])
Xtex.iloc[:, 4:atribute_numbers] = scaler.transform(Xtex.iloc[:, 4:atribute_numbers])

scaler = preprocessing.StandardScaler()
Xtry.iloc[:, 4:atribute_numbers] = scaler.fit_transform(Xtry.iloc[:, 4:atribute_numbers])
Xtey.iloc[:, 4:atribute_numbers] = scaler.transform(Xtey.iloc[:, 4:atribute_numbers])

# Clases y dummy para target variable

class_number = ytrx.unique().size

Xtrx = np.array(Xtrx,dtype='float32')
Xtex = np.array(Xtex,dtype='float32')

ytrx_cat = pd.Series(ytrx, dtype="category")
ytrx = np.array(ytrx_cat.cat.codes,dtype='int32')

In [4]:
feature_cols = [tf.contrib.layers.real_valued_column("", dimension=atribute_numbers)]

In [5]:
# Estimator using the default optimizer.
estimator = tf.contrib.learn.LinearClassifier(
    feature_columns=feature_cols,
    n_classes=3,
    model_dir="./tmp/linear_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A082935C50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './tmp/linear_model'}


In [6]:
# Define the training inputs
def get_train_inputs():
    x = tf.constant(Xtrx)
    y = tf.constant(ytrx)
    return x, y

In [7]:
estimator.fit(input_fn=get_train_inputs, steps=5000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./tmp/linear_model\model.ckpt.
INFO:tensorflow:loss = 1.09861, step = 1
INFO:tensorflow:global_step/sec: 928.92
INFO:tensorflow:loss = 0.729144, step = 101 (0.108 sec)
INFO:tensorflow:global_step/sec: 861.57
INFO:tensorflow:loss = 0.71199, step = 201 (0.116 sec)
INFO:tensorflow:global_step/sec: 1007.76
INFO:tensorflow:loss = 0.704808, step = 301 (0.115 sec)
INFO:tensorflow:global_step/sec: 801.969
INFO:tensorflow:loss = 0.701096, step = 401 (0.113 sec)
INFO:tensorflow:global_step/sec: 913.475
INFO:tensorflow:loss = 0.698973, step = 501 (0.109 sec)
INFO:tensorflow:global_step

LinearClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x000001A0810E5320>, 'feature_columns': [_RealValuedColumn(column_name='', dimension=8, default_value=None, dtype=tf.float32, normalizer=None)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})